In [1]:
from ckiptagger import WS, POS
import numpy as np
import pandas as pd
ws = WS("./data")
pos = POS("./data")

C:\Users\Xiaoyong\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\Users\Xiaoyong\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ckiptagger\model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


In [2]:
#讀取文件
def read_file(filename):
    with  open(filename, 'r',encoding='utf-8')as f:
        text = f.read()
        #回傳list
        text = text.split('\n')
    return text

In [3]:
#寫入文件
def write_data(filename,data):
    with open(filename,'a',encoding='utf-8')as f:
        f.write(str(data))


In [5]:
#句子斷詞
def tokenize(sentence):
    words = ws([sentence])
    print(words)
    return words


In [6]:
#詞性標注
def postagger(words):
    postags = pos(words)
    return postags


In [7]:
#斷詞和標注回傳成(TUPLE)
def intergrad_word(words,postags):
    #拉链算法，两两匹配
    pos_list = zip(words,postags)
    pos_list = [ w for w in pos_list]
    return pos_list

In [8]:
#去除停用詞
def del_stopwords(words):
    stopwords = read_file(r"C:\Users\Xiaoyong\Desktop\學期資料\111-2_course\NLP\final\停用詞表\cn_stopwords_tw.txt")
    new_words = []
    for word in words[0]:
        if word not in stopwords:
            new_words.append(word)
    return new_words

In [9]:
# 讀取程度副詞
def weighted_value(request):
    result_dict = []
    if request == "one":
        result_dict = read_file(r"C:\Users\Xiaoyong\Desktop\學期資料\111-2_course\NLP\final\情感辭典\chinese_sentiment_dictionary-master\file\情感词典\知网\繁體\程度級別\most.txt")
    elif request == "two":
        result_dict = read_file(r"C:\Users\Xiaoyong\Desktop\學期資料\111-2_course\NLP\final\情感辭典\chinese_sentiment_dictionary-master\file\情感词典\知网\繁體\程度級別\very.txt")
    elif request == "three":
        result_dict = read_file(r"C:\Users\Xiaoyong\Desktop\學期資料\111-2_course\NLP\final\情感辭典\chinese_sentiment_dictionary-master\file\情感词典\知网\繁體\程度級別\more.txt")
    elif request == "four":
        result_dict = read_file(r"C:\Users\Xiaoyong\Desktop\學期資料\111-2_course\NLP\final\情感辭典\chinese_sentiment_dictionary-master\file\情感词典\知网\繁體\程度級別\ish.txt")
    elif request == "five":
        result_dict = read_file(r"C:\Users\Xiaoyong\Desktop\學期資料\111-2_course\NLP\final\情感辭典\chinese_sentiment_dictionary-master\file\情感词典\知网\繁體\程度級別\insufficiently.txt")
    elif request == "six":
        result_dict = read_file(r"C:\Users\Xiaoyong\Desktop\學期資料\111-2_course\NLP\final\情感辭典\chinese_sentiment_dictionary-master\file\情感词典\知网\繁體\程度級別\inverse.txt")
    elif request == 'posdict':
        result_dict = read_file(r"C:\Users\Xiaoyong\Desktop\學期資料\111-2_course\NLP\final\情感辭典\chinese_sentiment_dictionary-master\file\情感词典\final_positive.txt")
    elif request == 'negdict':
        result_dict = read_file(r"C:\Users\Xiaoyong\Desktop\學期資料\111-2_course\NLP\final\情感辭典\chinese_sentiment_dictionary-master\file\情感词典\final_negative.txt")
    else:
        pass
    return result_dict

In [10]:
print("reading sentiment dict .......")
posdict = weighted_value('posdict')

negdict = weighted_value('negdict')
mostdict = weighted_value('one')
verydict = weighted_value('two')
moredict = weighted_value('three')
ishdict = weighted_value('four')
insufficientdict = weighted_value('five')
inversedict = weighted_value('six')

reading sentiment dict .......


In [11]:
#程度副詞權重
def match_adverb(word,sentiment_value):
    #最高級權重
    if word in mostdict:
        print('A')
        sentiment_value *= 8
    #比較級權重
    elif word in verydict:
        print('B')
        sentiment_value *= 6
    #比較級權重
    elif word in moredict:
        print('C')
        sentiment_value *= 4
    #輕微級權重
    elif word in ishdict:
        print('D')
        sentiment_value *= 2
    #相對程度
    elif word in insufficientdict:
        print('E')
        sentiment_value *= 0.5
    #否定詞權重
    elif word in inversedict:
        print('F')
        sentiment_value *= -1
    else:
        sentiment_value *= 1
    return sentiment_value

In [17]:

#对每一条微博打分
def single_sentiment_score(words,seg_words):
    
    #i，s 记录情感词和程度词出现的位置
    i = 0   #记录扫描到的词位子
    s = 0   #记录情感词的位置
    poscount = 0 #记录积极情感词数目
    negcount = 0 #记录消极情感词数目
    
    #逐个查找情感词
    for word in seg_words:
        #如果为积极词
        # print('我是:'+word)

        if word in posdict:
            # print('積極:'+word)
            
            poscount += 1  #情感词数目加1
        #在情感词前面寻找程度副词
            for w in seg_words[s:i]:
                poscount = match_adverb(w,poscount)
            s = i+1 #记录情感词位置
        # 如果是消极情感词
        elif word in negdict:
            # print('消極:'+word)
            # print(word)
            negcount +=1
            for w in seg_words[s:i]:
                negcount = match_adverb(w,negcount)
            s = i+1
        elif word =='!' or  word =='！' or word =='?' or word == '？':
            for w2 in seg_words[i::-1]:
                #如果为积极词，poscount+2
                if w2 in posdict:
                    poscount += 4
                    break
                #如果是消极词，negcount+2
                elif w2 in negdict:
                    negcount += 4
                    break
        i += 1 #定位情感词的位置
    #计算情感值
    print('最終分數')
    print(poscount,negcount)
    sentiment_score = poscount - negcount
    print('總分:'+str(sentiment_score))
    if sentiment_score >= 7:
        sentiment_score = 2
    elif sentiment_score > 0:
        sentiment_score = 1
    elif sentiment_score == 0:
        sentiment_score = 0
    elif sentiment_score < 7:
        sentiment_score = -1
    else:
        sentiment_score = -2
        
    return sentiment_score

In [12]:
def run_score(contents):
    scores_list = []
    for content in contents:
        content = content.strip()
        if content !='':
            words = tokenize(content)
            seg_words = del_stopwords(words)
            score = single_sentiment_score(words,seg_words) 
            content = '|'.join(seg_words)
            scores_list.append((content, score)) 
    return scores_list

In [18]:
df = pd.read_csv(r'C:\Users\Xiaoyong\Desktop\學期資料\111-2_course\NLP\final\simplifyweibo_4_moods\traditional_weibo.csv')
sentences = df['Reviews'].tolist()
print(len(sentences))

KeyError: 'Reviews'

In [22]:
#主程式
if __name__ == '__main__':

    # with open(r'C:\Users\Xiaoyong\Desktop\學期資料\111-2_course\NLP\final\simplifyweibo_4_moods\traditional_weibo.csv', 'r', encoding='utf-8') as file:
    #     sentences = file.readlines()
    
    sentences = ['何等幸運的我撿到五千塊!即使午餐有點難吃，也無所謂了']
    # df = pd.read_csv('google_37k.csv')
    # sentences = df['Reviews'].tolist()

    scores = run_score(sentences)
    print(scores)
    al_sentiment = []
    for score in scores:
        print('情感分值：',score[1])
        if score[1] < 0:
            print('情感:消極')
            s = '消級'
        elif score[1] == 0:
            print('情感:中性')
            s = '中性'
        else:
            print('情感:積極')
            s = '積極'
        al_sentiment.append(s)
        print('情感分析文本：',score[0])
    i = 0
    #寫入文件
    df = pd.DataFrame(scores, columns=['content', 'score'])
    df.to_csv('weibo_train_50k.csv', index=False)
    filename = r'train_data.txt'
    for score in scores:
        write_data(filename, '情感分析文本：{}'.format(str(score[0]))+'\n') 
        write_data(filename,'情感分值：{}'.format(str(score[1]))+'\n') 
        write_data(filename,'\n')
        i +=1
    print('succeed.......')

[['何等', '幸運', '的', '我', '撿到', '五千', '塊', '!', '即使', '午餐', '有點', '難吃', '，', '也', '無所謂', '了']]
B
D
最終分數
10 3
總分:7
[('何等|幸運|撿到|五千|塊|!|午餐|有點|難吃|無所謂', 2)]
情感分值： 2
情感倾向：积极
情感分析文本： 何等|幸運|撿到|五千|塊|!|午餐|有點|難吃|無所謂
succeed.......


In [ ]:
df = pd.read_csv('weibo_train_50k.csv')
df.dropna(inplace=True)
print(df.isnull().sum())
df.to_csv('weibo_train_50k_dropna.csv')

content    0
score      0
dtype: int64
